### Carlos Morán y Carlos Tardón grupo 9

# Práctica 1.B Problema 3

Un problema que queremos resolver es obtener un número objetivo dado a partir de la combinación de números primos mediante sumas, multiplicaciones y divisiones, a excepción de un primo prohibido (dado) que no se podrá utilizar. De las posibles soluciones existentes para dos números dados (objetivo y prohibido) serán mejores aquellas soluciones que utilicen el menor número de factores primos posibles. Los números primos a utilizar podrán ser aquellos de una o dos cifras, incluido el uno.

### Interpretación del enunciado

Al igual que en el problema 2, hemos considerado que todas las operaciones sean asociativas por la izquierda, de tal manera que 2 + 3 * 5 = (2+3)*5 = 25. Esto permite que el estado sea simplemente el acumulado de las operaciones que ya llevamos, y para realizar una nueva operacion, la realizamos sobre el acumulado.  

### Representación de Estados

Una entero que indique el número por el que vamos. Por tanto, el estado inicial es 0 y el objetivo coincide con el numero al que queremos llegar.


### Representación de Acciones
Las acciones se pueden representar como una pareja (op,num) con num siendo el nuevo número primo que estamos añadiendo y op la nueva operación que estamos haciendo (suma, resta, multiplicación, división).

### Coste
Minimizaremos la longitud de la solución, es decir, el número de primos y operaciones usados

### Heurística
Podríamos pensar que si minimizamos el valor absoluto de la diferencia con el estado objetivo, entonces llegaremos antes al resultado requerido. Por tanto, podemos usar abs(self.goal - node.state) como heurística. Esta heurística no va a ser admisible, pues no intenta hallar un coste aproximado, tan solo proporciona un valor que guíe al algoritmo de búsqueda para que se acerque a la solución. Además, puede hacer que el número de nodos analizados sea mayor que si no hubieramos usado esta heuristica. Pese a estas complicaciones, hemos elegido implementar esta heurística pues, en el caso de que el número final esté muy alejado del inicial (normalmente el 0), esta heurística nos acercará más rápidamente al objetivo.

Otra posible heurística sería relajar las condiciones y permitir el uso de todos los factores primos. De esta manera, en cada nodo se descompone en factores primos la diferencia en valor absoluto de lo que llevamos menos el estado final, y nuestra heurística es el número de factores primos(contados con su multiplicidad). Esta heurística es contraproducente, pues en cada nodo tiene que resolver un problema(la descomposición en primos) que en el caso general es intratable. Dejamos constancia de esta idea, pero no la abordaremos.

In [5]:
import math
from search import *
import itertools

def opera(op1,sig,op2):

    if sig == '+':
        return op1 + op2
    elif sig == '-':
        return op1 - op2
    elif sig == '*':
        return op1 * op2
    else:
        return op1 / op2

def sieve(n):
    nums = list(range(1,n+1))
    interval = range(2,math.trunc(math.sqrt(n))+1)

    for i in interval:
        m = 2*i
        while m <= n:
            if nums.count(m) > 0:
                nums.remove(m)
            m += i
    return nums

In [6]:
class Prohibido(Problem):

    def __init__(self, goal, ban, initial = 0):
        self.analizados  = 0
        self.primes = sieve(100)
        self.prim = self.primes[1:]
        if self.primes.count(ban) > 0:
            self.primes.remove(ban)
        Problem.__init__(self,initial, goal)

    def actions(self,estado):
        ## El conjunto de hijos es, en principio, el conjunto producto del conjunto de símbolos (+,-,*,/) con los primos del 1 al 100 
        accs = list(itertools.product(*[('+','-','*','/'),tuple(self.primes)]))

        return accs

    def result(self,estado,accion):
        return opera(estado, accion[0], accion[1])
        
    def goal_test(self,estado):
        self.analizados += 1
        return self.goal == estado
    
    ## heuristica por defecto. Tiene que estar definida para poder usar busqueda bidireccional
    def h(self,node):
        return 0
  
    ## Heuristica que hemos elegido
    def heuristica(self, node):
        return abs(self.goal - node.state)
    
    ## Heuristica fallida. Intenta aproximar el coste descomponiendo en factores primos 
    def h2(self, node):
        num = abs(self.goal - node.state)
        num_fact = 0
        while(num > 1):
            for p in self.prim:
                if p > num:
                    break
                if num % p == 0:
                    num = num/p
                    num_fact += 1
        return num_fact

In [7]:
def resuelve_prohibido(problem, algoritmo, h=None , print_sol=True):
    if h: 
        res = algoritmo(problem,h)
    else:
        res = algoritmo(problem)
    if res is None:
        print("Instancia sin solución")
    else:
        sol = res.solution()
        if print_sol: #Para calcular tiempos, no printeamos la solución
            print("Solución: {0}".format(sol))
            print("Nodos analizados: {0}".format(problem.analizados))
            if h:
                print("Heurística: {0}".format(h.__name__))
            print("Algoritmo: {0}".format(algoritmo.__name__))

In [7]:
# Se puede hacer busqueda bidireccional, que devuelve la longitud del camino más corto (que en este caso es 3)
I1 = Prohibido(123,3)
print(bidirectional_search(I1))

3


In [12]:
I1 = Prohibido(101,2)
resuelve_prohibido(I1,breadth_first_tree_search, print_sol=True)

Solución: [('+', 1), ('+', 3), ('+', 97)]
Nodos analizados: 10226
Algoritmo: breadth_first_tree_search


Como el estado inicial es 0, siempre se aplica una operación inicial. Como la búsqueda es en anchura, esta primera operación no es '/' ni '*', (pues se llegaría al 0 de nuevo, y por tanto se ejecutarían acciones de mas), así que solo puede ser suma '+', que representa que el primer primo es positivo, o resta '-', que representa que es negativo

In [11]:
%%timeit
I1 = Prohibido(101,2)
resuelve_prohibido(I1,breadth_first_tree_search, print_sol=False)

1.83 s ± 327 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
I1 = Prohibido(101,2)
resuelve_prohibido(I1,best_first_graph_search,I1.heuristica, print_sol=True)

Solución: [('+', 97), ('+', 3), ('+', 1)]
Nodos analizados: 4
Heurística: heuristica
Algoritmo: best_first_graph_search


In [16]:
%%timeit
I1 = Prohibido(101,2)
resuelve_prohibido(I1,best_first_graph_search,I1.heuristica, print_sol=False)

13.9 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Como se ve, la heuristica reduce rapidamente la distancia con el objetivo, y por tanto podemos usar busqueda voraz para llegar a una solución, incluso si el numero objetivo es muy alto:

In [15]:
# Caso alto sin heuristica
I1 = Prohibido(500,11)
resuelve_prohibido(I1,breadth_first_graph_search, print_sol=True)

Solución: [('+', 3), ('+', 97), ('*', 5)]
Nodos analizados: 9351
Algoritmo: breadth_first_graph_search


In [16]:
# Caso alto con heuristica
I1 = Prohibido(500,2)
resuelve_prohibido(I1,best_first_graph_search,I1.heuristica, print_sol=True)

Solución: [('+', 97), ('*', 5), ('+', 13), ('+', 1), ('+', 1)]
Nodos analizados: 6
Heurística: heuristica
Algoritmo: best_first_graph_search


In [21]:
# Caso más alto con heuristica(aqui anchura es inviable)
I1 = Prohibido(90001,2)
resuelve_prohibido(I1,best_first_graph_search,I1.heuristica, print_sol=True)

Solución: [('+', 97), ('*', 97), ('*', 11), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97),

In [12]:
# Caso aun mas alto con heuristica
I1 = Prohibido(400001,2)
resuelve_prohibido(I1,best_first_graph_search,I1.heuristica, print_sol=True)

Solución: [('+', 97), ('*', 97), ('*', 43), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 97), ('-', 29), ('+', 1), ('+', 1)]
Nodos analizados: 54
Heurística: heuristica
Algoritmo: best_first_graph_search


#### En el siguiente caso, la heurística reduce bastante el número de nodos recorridos, pero no llega a la solución óptima, pues no es admisible:

In [23]:
I2 = Prohibido(123,3) 
resuelve_prohibido(I2,breadth_first_tree_search, print_sol=True)

Solución: [('+', 1), ('+', 2), ('*', 41)]
Nodos analizados: 10264
Algoritmo: breadth_first_tree_search


In [25]:
I2 = Prohibido(123,3) 
resuelve_prohibido(I2,best_first_graph_search,I2.heuristica, print_sol=True)

Solución: [('+', 97), ('+', 23), ('+', 2), ('+', 1)]
Nodos analizados: 5
Heurística: heuristica
Algoritmo: best_first_graph_search


In [69]:
# Prueba con negativo
I2 = Prohibido(-123,3) 
resuelve_prohibido(I2,best_first_graph_search,I2.heuristica, print_sol=True)

Solución: [('-', 97), ('-', 29), ('+', 2), ('+', 1)]
Nodos analizados: 5
Heurística: heuristica
Algoritmo: best_first_graph_search


In [71]:
I1 = Prohibido(-123,3)
resuelve_prohibido(I1,breadth_first_tree_search, print_sol=True)

Solución: [('+', 1), ('-', 41), ('-', 83)]
Nodos analizados: 13849
Algoritmo: breadth_first_tree_search


### Posibles complicaciones
<ul>
    <li> Podrían prohibirnos alguna operación. En ese caso, se eliminaría de las operaciones disponibles en la función __init__</li>
    <li> Podrían restringirnos el uso del mismo primo en dos acciones consecutivas (es decir, 2+2=4 estaría prohibido). En ese caso, habría que guardar en el estado el primo usado anteriormente, y en la función actions se eliminaría ese primo de los primos a usar.</li>
    <li> Podrían aumentar los primos que podemos usar, por ejemplo, que solo podamos usar primos entre el 0 y el 50. En ese caso, habría que cambiar la llamada a la criba de Erathostenes.</li>
    <li> Podrían prohibirnos usar más de un número. En cuyo caso, habría que almacenar todos los números prohibidos en una tupla y, mediante diferencia de conjuntos, quitar todos los números que estén en esa tupla de los primos disponibles (obtenidos en la criba de Erathostenes)</li>
    <li>Una generalización de este problema sería permitir el uso de paréntesis, y por tanto poder cambiar la asociatividad de las operaciones. En ese caso habría que enfocar la búsqueda de otra forma diferente, mediante el uso de árboles sintácticos</li>
<ul>